<a href="https://colab.research.google.com/github/Darkdew/RFRandomizedSearchCV/blob/master/RF_RandomizedSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df= pd.read_csv('/content/drive/My Drive/Colab Notebooks/RF_RandomizedSearchCV/Social_Network_Ads.csv')
df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [0]:
X= df.iloc[:, [2,3]].values
y= df.iloc[:, 4].values

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=7)

In [0]:
from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [6]:
#fitting RandomForestClassifier

from sklearn.ensemble import RandomForestClassifier

RFClassifier= RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=50)
RFClassifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=50, verbose=0,
                       warm_start=False)

In [0]:
y_pred= RFClassifier.predict(X_test)

In [11]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm= confusion_matrix(y_test, y_pred)
cm
score= accuracy_score(y_test, y_pred)
print(score)

0.8875


In [0]:
#but how did we decide on the parameter values of our RandomForestClassifier?
#this can be done by RandomizedSearchCV

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [0]:
estimator= RandomForestClassifier(n_jobs=-1)

rf_param_dist= { 'max_depth' : [3,5,7, 10, None],
                'n_estimators': [100,200,300,400,500],
                'max_features' : randint(1,3),
                'criterion': ['gini','entropy'],
                'bootstrap' : [True, False],
                'min_samples_leaf': randint(1,4)
    
                }


def hptuning_RSCV(estimator, parameteric_distribution, iteration_count, X_train, y_train):
  randomizedsearchcv = RandomizedSearchCV(estimator, 
                                          param_distributions=parameteric_distribution, 
                                          n_jobs=-1,
                                          n_iter= iteration_count,
                                          cv=10
                                          )
  randomizedsearchcv.fit(X_train, y_train)
  best_params= randomizedsearchcv.best_params_
  best_score= randomizedsearchcv.best_score_
  
  return best_params, best_score



In [0]:
best_parameters, best_scoring= hptuning_RSCV(estimator, rf_param_dist, 35, X_train, y_train)

In [17]:
best_parameters

{'bootstrap': True,
 'criterion': 'gini',
 'max_depth': 3,
 'max_features': 2,
 'min_samples_leaf': 3,
 'n_estimators': 300}

In [18]:
best_scoring

0.90625

In [19]:
#Using these best parametric values, lets rebuild our Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

RFClassifier_final= RandomForestClassifier(n_estimators=300, 
                                           criterion='gini', 
                                           random_state=50, 
                                           bootstrap= True,
                                           min_samples_leaf= 3,
                                           max_depth=3,
                                           max_features=2
                                           )
RFClassifier_final.fit(X_train, y_train)

y_pred= RFClassifier_final.predict(X_test)
score= accuracy_score(y_test, y_pred)
score

0.925

In [21]:
#Lets run cross_val_score on our datset and check the score

from sklearn.model_selection import cross_val_score

cross_val= cross_val_score(RFClassifier_final, X, y, cv=10, scoring= 'accuracy')
cross_val

array([1.   , 0.65 , 0.9  , 1.   , 1.   , 0.85 , 0.825, 0.775, 0.875,
       0.975])

In [22]:
cross_val.mean()

0.885